In [1]:
from urllib.request import urlopen

from bs4 import BeautifulSoup
from statistics import mean 
import pandas as pd
import re
from datetime import datetime,timedelta 


import requests


import warnings
warnings.filterwarnings("ignore")

In [2]:
#дата сегодня
date_now=datetime.today().date()
day_now=date_now.day #<--- день сегодня
date_tomorrow=date_now + timedelta(days=1)
day_tomorrow=(date_now + timedelta(days=1)).day #<--- день завтра
day_after_tomorrow=(date_now + timedelta(days=2)).day #<--- день послезавтра
day_4=(date_now + timedelta(days=3)).day #<--- 4-ый
day_5=(date_now + timedelta(days=4)).day #<--- 5-ый
day_6=(date_now + timedelta(days=5)).day #<--- 6-ый
day_7=(date_now + timedelta(days=6)).day #<--- 7-ый


date_after_tomorrow=(date_now + timedelta(days=2))
date_4=(date_now + timedelta(days=3)) #<--- 4-ый
date_5=(date_now + timedelta(days=4)) #<--- 5-ый
date_6=(date_now + timedelta(days=5)) #<--- 6-ый
date_7=(date_now + timedelta(days=6)) #<--- 7-ый
date_8=(date_now + timedelta(days=7)) #<--- 8-ый

#время сегодня
time_now=datetime.today().time().hour

In [3]:
#считываем страницу с прогнозом Яндекса через read_html, получаем необработанные данные 
ya_df=pd.read_html('https://yandex.ru/pogoda/details?lat=55.753215&lon=37.622504&via=ms')


In [4]:
#каждый день находится в отдельной ячейке, прилепим их их друг под другом

ya_new=pd.concat([ya_df[0],ya_df[1],ya_df[2],ya_df[3],ya_df[4],ya_df[5],ya_df[6],ya_df[7],ya_df[8],ya_df[9]],ignore_index=True)
ya_new.columns=['temp','nan','weather','press','wet','wind','t_eff']

## ЯНДЕКС

In [5]:
#посмотрим, что получилось
ya_new.head(20)

,temp,nan,weather,press,wet,wind,t_eff
0,"утром+3…+8утром, от 3 до 8°",NaN,Облачно с прояснениями,752,84%,"3,5СВ4, северо-восточный",+22°
1,"днём+10…+12днём, от 10 до 12°",NaN,Пасмурно,752,61%,"4,3СВ4, северо-восточный",+77°
2,"вечером+8…+11вечером, от 8 до 11°",NaN,Пасмурно,752,70%,"3,0СВ3, северо-восточный",+77°
3,"ночью+3…+7ночью, от 3 до 7°",NaN,Пасмурно,754,80%,"3,5СВ4, северо-восточный",+11°
4,"утром+3…+6утром, от 3 до 6°",NaN,Пасмурно,755,81%,"4,7СВ5, северо-восточный",00°
5,"днём+8…+11днём, от 8 до 11°",NaN,Облачно с прояснениями,755,57%,"5,0СВ5, северо-восточный",+55°
6,"вечером+6…+10вечером, от 6 до 10°",NaN,Ясно,755,59%,"3,8СВ4, северо-восточный",+33°
7,"ночью0…+5ночью, от 0 до 5°",NaN,Ясно,755,72%,"3,7СВ4, северо-восточный",−2-2°
8,"утром0…+5утром, от 0 до 5°",NaN,Облачно с прояснениями,756,70%,"4,7В5, восточный",−3-3°
9,"днём+6…+9днём, от 6 до 9°",NaN,Облачно с прояснениями,756,49%,"5,0В5, восточный",+33°


In [6]:
temp=ya_new['temp'].str.rsplit(',',expand=True).drop(1,axis=1)

In [7]:
#поработаем со столбцом температуры
temp.head()

,0
0,утром+3…+8утром
1,днём+10…+12днём
2,вечером+8…+11вечером
3,ночью+3…+7ночью
4,утром+3…+6утром


In [8]:
#убираем из температуры слова

temp[0]=temp[0].str.replace('утром','')
temp[0]=temp[0].str.replace('днём','')
temp[0]=temp[0].str.replace('вечером','')
temp[0]=temp[0].str.replace('ночью','')



temp_full=temp[0].str.rsplit('…',expand=True)
temp_full.fillna('-999',inplace=True)
temp_full

for i in range(0,len(temp_full.index)):
    if temp_full[1][i]=='-999':
        temp_full[1][i]=temp_full[0][i]

temp_full.columns=['t_down','t_up']        

temp_full.head() # <--- тут лежит файл с температурой Яндекса

,t_down,t_up
0,+3,+8
1,+10,+12
2,+8,+11
3,+3,+7
4,+3,+6


In [9]:
display(ya_new.head(),ya_new.tail())

,temp,nan,weather,press,wet,wind,t_eff
0,"утром+3…+8утром, от 3 до 8°",NaN,Облачно с прояснениями,752,84%,"3,5СВ4, северо-восточный",+22°
1,"днём+10…+12днём, от 10 до 12°",NaN,Пасмурно,752,61%,"4,3СВ4, северо-восточный",+77°
2,"вечером+8…+11вечером, от 8 до 11°",NaN,Пасмурно,752,70%,"3,0СВ3, северо-восточный",+77°
3,"ночью+3…+7ночью, от 3 до 7°",NaN,Пасмурно,754,80%,"3,5СВ4, северо-восточный",+11°
4,"утром+3…+6утром, от 3 до 6°",NaN,Пасмурно,755,81%,"4,7СВ5, северо-восточный",00°


,temp,nan,weather,press,wet,wind,t_eff
35,"ночью+2…+4ночью, от 2 до 4°",NaN,Облачно с прояснениями,753,80%,"2,2В2, восточный",−1-1°
36,"утром+2…+5утром, от 2 до 5°",NaN,Пасмурно,752,81%,"2,8В3, восточный",00°
37,"днём+8…+11днём, от 8 до 11°",NaN,Облачно с прояснениями,751,57%,"4,0ЮВ4, юго-восточный",+66°
38,"вечером+9…+10вечером, от 9 до 10°",NaN,Облачно с прояснениями,751,65%,"3,6СВ4, северо-восточный",+66°
39,"ночью+6…+8ночью, от 6 до 8°",NaN,Пасмурно,751,81%,"3,0С3, северный",+44°


In [10]:
ya_new['wind']=ya_new['wind'].str.replace('Ю','*')
ya_new['wind']=ya_new['wind'].str.replace('С','*')
ya_new['wind']=ya_new['wind'].str.replace('З','*')
ya_new['wind']=ya_new['wind'].str.replace('В','*')
ya_new['wind']=ya_new['wind'].str.replace('ШтильШтиль','0')
ya_new['wind']=ya_new['wind'].str.replace(',','.')

wind=ya_new['wind'].str.rsplit('*',expand=True)
wind.drop([1],axis=1,inplace=True)
wind.drop([2],axis=1,inplace=True)
wind.columns=['wind']
wind.head()

,wind
0,3.5
1,4.3
2,3.0
3,3.5
4,4.7


In [11]:
t_eff=[]
for i in range(0,ya_new.shape[0]):
    string=ya_new['t_eff'][i]
    if string[0]=='−':
        s_1=re.sub('[^0-9]','', string)
        #end=(len(s_1)/2)
        s_fin='−'+s_1[:len(s_1)//2]
    if string[0] != '−':  
        s_1=re.sub('[^0-9]','', string)
        s_fin=s_1[:len(s_1)//2]
    t_eff.append(s_fin)
    
temp_eff=pd.Series(t_eff) 


In [12]:
#список утро-день-ночь
list_days=[10*['утро','день','вечер','ночь']]
list_days=pd.DataFrame(list_days).T


In [13]:
ya_new['wet']=ya_new['wet'].str.replace('%','')

ya_full=pd.concat([list_days,temp_full,ya_new[['press','wet']],wind,temp_eff,ya_new['weather']],axis=1)

ya_full.columns=['time','t_down','t_up','press','wet','wind','t_eff','weather']

ya_full['t_down']=ya_full['t_down'].str.replace('−','-')
ya_full['t_up']=ya_full['t_up'].str.replace('−','-')

ya_full['t_eff']=ya_full['t_eff'].str.replace('−','-')

ya_full.head(5)

,time,t_down,t_up,press,wet,wind,t_eff,weather
0,утро,+3,+8,752,84,3.5,2,Облачно с прояснениями
1,день,+10,+12,752,61,4.3,7,Пасмурно
2,вечер,+8,+11,752,70,3.0,7,Пасмурно
3,ночь,+3,+7,754,80,3.5,1,Пасмурно
4,утро,+3,+6,755,81,4.7,0,Пасмурно


In [14]:
ya_full.head()

,time,t_down,t_up,press,wet,wind,t_eff,weather
0,утро,+3,+8,752,84,3.5,2,Облачно с прояснениями
1,день,+10,+12,752,61,4.3,7,Пасмурно
2,вечер,+8,+11,752,70,3.0,7,Пасмурно
3,ночь,+3,+7,754,80,3.5,1,Пасмурно
4,утро,+3,+6,755,81,4.7,0,Пасмурно


In [15]:
ya_full[['t_down','t_up','wet']]=ya_full[['t_down','t_up','wet']].astype('int')
ya_full['wind']=ya_full['wind'].astype('float')


ya_full['t_eff']=ya_full['t_eff'].astype('int')
#считаем среднюю температуру
ya_full['t_mean']=(ya_full['t_up']+ya_full['t_down'])/2

In [16]:
ya_full.head(5)

,time,t_down,t_up,press,wet,wind,t_eff,weather,t_mean
0,утро,3,8,752,84,3.5,2,Облачно с прояснениями,5.5
1,день,10,12,752,61,4.3,7,Пасмурно,11.0
2,вечер,8,11,752,70,3.0,7,Пасмурно,9.5
3,ночь,3,7,754,80,3.5,1,Пасмурно,5.0
4,утро,3,6,755,81,4.7,0,Пасмурно,4.5


In [17]:
ya_full.info() #<--- EDA Яндекс готов

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   time     40 non-null     object 
 1   t_down   40 non-null     int64  
 2   t_up     40 non-null     int64  
 3   press    40 non-null     int64  
 4   wet      40 non-null     int64  
 5   wind     40 non-null     float64
 6   t_eff    40 non-null     int64  
 7   weather  40 non-null     object 
 8   t_mean   40 non-null     float64
dtypes: float64(2), int64(5), object(2)
memory usage: 2.9+ KB
